In [25]:
import numpy as np
import pandas as pd
from collections import Counter
from collections import OrderedDict
from gensim.models import word2vec
import re
import pickle
import MeCab
import tensorflow as tf

In [2]:
All_df = pd.read_pickle("../data/all_mecab.pickle")
All_df

,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,21,21,31,31,40,40,50,50
1,"[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,21,21,31,31,40,40,50,50
2,"[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,21,21,31,31,40,40,50,50
3,"[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,21,21,31,31,40,40,52,52
4,"[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,22,22,32,32,41,40,50,50
5,"[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,22,22,32,32,41,40,50,50
6,"[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,22,22,32,32,42,42,50,50
7,"[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,23,23,32,32,41,41,50,50
8,"[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,25,25,32,32,41,41,50,50
9,"[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,21,22,31,32,40,41,50,50


In [59]:
test = All_df.values[:5,0]
print("test",test,"\n")
print(Counter([len(t) for t in All_df.values[:,0]]))

test [list(['よろしく', 'お願い', 'し', 'ます', '！', 'EOS'])
 list(['よろしく', 'お願い', 'し', 'ます', 'EOS'])
 list(['名前', 'な', 'の', 'が', '恥ずかしい', 'です', 'EOS', 'よろしく', 'お願い', 'し', 'ます', '！', 'EOS'])
 list(['早速', '課題', 'やっ', 'ちゃ', 'い', 'ましょ', 'う', '！', 'EOS'])
 list(['やっ', 'ちゃ', 'い', 'ましょ', 'う', 'EOS', 'ｍｏｏｄｌｅ', 'は', 'ゴミ', 'です', '！', 'EOS'])] 

Counter({5: 1087, 6: 874, 4: 782, 7: 659, 8: 631, 3: 590, 10: 525, 9: 524, 11: 506, 13: 446, 12: 435, 14: 406, 2: 396, 15: 342, 16: 285, 17: 276, 19: 232, 18: 231, 20: 205, 21: 199, 1: 181, 22: 180, 23: 156, 24: 141, 25: 132, 26: 125, 27: 114, 28: 113, 29: 86, 31: 83, 33: 77, 30: 69, 32: 67, 34: 62, 38: 59, 35: 58, 37: 48, 39: 46, 36: 45, 41: 36, 43: 36, 40: 32, 47: 31, 44: 29, 45: 27, 42: 26, 48: 24, 52: 18, 49: 17, 46: 16, 50: 14, 56: 12, 55: 12, 51: 11, 59: 11, 54: 11, 80: 10, 53: 9, 57: 9, 62: 8, 65: 7, 58: 7, 64: 6, 66: 6, 61: 6, 60: 5, 71: 5, 63: 4, 87: 4, 73: 4, 85: 4, 76: 4, 74: 4, 89: 3, 88: 3, 69: 3, 97: 3, 110: 3, 92: 2, 82: 2, 79: 2, 95: 2, 91: 2, 78: 

In [62]:
model = word2vec.Word2Vec.load("../data/w2v_wo_wiki.model")
model['よろしく']
# for keys,values in model.wv.vocab.items():
#     print(keys)
#     print(values)

/Users/kinten/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


KeyError: "word 'よろしく' not in vocabulary"

In [3]:
'''
test = All_df.values[:5,0]
print("test",test,"\n")


data = [x for s in test for x in s]
wd_set = Counter(data)
print("data",data)
print("wd_set",wd_set,"\n")



wd_cp = sorted(wd_set.items(), key=lambda x: (-x[1], x[0]))
print("単語頻度辞書",wd_cp,"\n")
wd_ary, wd_cnt = list(zip(*wd_cp))
print("単語リスト",wd_ary,"\n")
print("単語出現数リスト",wd_cnt,"\n")
wd_to_id = dict(zip(wd_ary, range(len(wd_ary))))
print("単語ID辞書",wd_to_id,"\n")

maxlen = max([len(t) for t in test])

sen_id = []
for s in test:
    sid = [wd_to_id[wd] for wd in s]
    sen_id.append(np.pad(sid,(0,maxlen-len(sid)),'constant'))
print(sen_id)


# print("test[0]をIDに",np.pad(wd_to_id[test[0]],(0,maxlen-len(test[0])),'constant'))
vocabulary = len(wd_to_id)
print(vocabulary)



tf.reset_default_graph()


embedding = tf.get_variable("embedding",[vocabulary,3])
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
print(tf.nn.embedding_lookup(embedding,sen_id[0]).eval())
print(tf.nn.embedding_lookup(embedding,sen_id[1]).eval())
'''

'\ntest = All_df.values[:5,0]\nprint("test",test,"\n")\n\n\ndata = [x for s in test for x in s]\nwd_set = Counter(data)\nprint("data",data)\nprint("wd_set",wd_set,"\n")\n\n\n\nwd_cp = sorted(wd_set.items(), key=lambda x: (-x[1], x[0]))\nprint("単語頻度辞書",wd_cp,"\n")\nwd_ary, wd_cnt = list(zip(*wd_cp))\nprint("単語リスト",wd_ary,"\n")\nprint("単語出現数リスト",wd_cnt,"\n")\nwd_to_id = dict(zip(wd_ary, range(len(wd_ary))))\nprint("単語ID辞書",wd_to_id,"\n")\n\nmaxlen = max([len(t) for t in test])\n\nsen_id = []\nfor s in test:\n    sid = [wd_to_id[wd] for wd in s]\n    sen_id.append(np.pad(sid,(0,maxlen-len(sid)),\'constant\'))\nprint(sen_id)\n\n\n# print("test[0]をIDに",np.pad(wd_to_id[test[0]],(0,maxlen-len(test[0])),\'constant\'))\nvocabulary = len(wd_to_id)\nprint(vocabulary)\n\n\n\ntf.reset_default_graph()\n\n\nembedding = tf.get_variable("embedding",[vocabulary,3])\nsess = tf.InteractiveSession()\ntf.global_variables_initializer().run()\nprint(tf.nn.embedding_lookup(embedding,sen_id[0]).eval())\nprint(t

In [4]:
'''
vocab = ['the','like','between','did','just','national','day','country','under','such','second']
emb = np.array([[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -0.044457, -0.49688, -0.17862],
   [0.36808, 0.20834, -0.22319, 0.046283, 0.20098, 0.27515, -0.77127, -0.76804],
   [0.7503, 0.71623, -0.27033, 0.20059, -0.17008, 0.68568, -0.061672, -0.054638],
   [0.042523, -0.21172, 0.044739, -0.19248, 0.26224, 0.0043991, -0.88195, 0.55184],
   [0.17698, 0.065221, 0.28548, -0.4243, 0.7499, -0.14892, -0.66786, 0.11788],
   [-1.1105, 0.94945, -0.17078, 0.93037, -0.2477, -0.70633, -0.8649, -0.56118],
   [0.11626, 0.53897, -0.39514, -0.26027, 0.57706, -0.79198, -0.88374, 0.30119],
   [-0.13531, 0.15485, -0.07309, 0.034013, -0.054457, -0.20541, -0.60086, -0.22407],
   [ 0.13721, -0.295, -0.05916, -0.59235, 0.02301, 0.21884, -0.34254, -0.70213],
   [ 0.61012, 0.33512, -0.53499, 0.36139, -0.39866, 0.70627, -0.18699, -0.77246 ],
   [ -0.29809, 0.28069, 0.087102, 0.54455, 0.70003, 0.44778, -0.72565, 0.62309 ]])
tf_embedding = tf.constant(emb, dtype=tf.float32)
embedding = tf.get_variable("embedding",[10000,20],dtype=tf.float32)
input_str = "like the country"
word_to_idx = OrderedDict({w:vocab.index(w) for w in input_str.split() if w in vocab})

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
print(list(word_to_idx.values()))
tf.nn.embedding_lookup(embedding, list(word_to_idx.values())).eval()
'''

'\nvocab = [\'the\',\'like\',\'between\',\'did\',\'just\',\'national\',\'day\',\'country\',\'under\',\'such\',\'second\']\nemb = np.array([[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -0.044457, -0.49688, -0.17862],\n   [0.36808, 0.20834, -0.22319, 0.046283, 0.20098, 0.27515, -0.77127, -0.76804],\n   [0.7503, 0.71623, -0.27033, 0.20059, -0.17008, 0.68568, -0.061672, -0.054638],\n   [0.042523, -0.21172, 0.044739, -0.19248, 0.26224, 0.0043991, -0.88195, 0.55184],\n   [0.17698, 0.065221, 0.28548, -0.4243, 0.7499, -0.14892, -0.66786, 0.11788],\n   [-1.1105, 0.94945, -0.17078, 0.93037, -0.2477, -0.70633, -0.8649, -0.56118],\n   [0.11626, 0.53897, -0.39514, -0.26027, 0.57706, -0.79198, -0.88374, 0.30119],\n   [-0.13531, 0.15485, -0.07309, 0.034013, -0.054457, -0.20541, -0.60086, -0.22407],\n   [ 0.13721, -0.295, -0.05916, -0.59235, 0.02301, 0.21884, -0.34254, -0.70213],\n   [ 0.61012, 0.33512, -0.53499, 0.36139, -0.39866, 0.70627, -0.18699, -0.77246 ],\n   [ -0.29809, 0.28069, 0.087102, 0.544

In [5]:
'''
embedding = tf.get_variable("embedding",initializer=tf.ones(shape=[10, 5]),dtype=tf.float32)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.nn.embedding_lookup(embedding, [1, 2, 3, 4]).eval()
'''

'\nembedding = tf.get_variable("embedding",initializer=tf.ones(shape=[10, 5]),dtype=tf.float32)\nsess = tf.InteractiveSession()\ntf.global_variables_initializer().run()\ntf.nn.embedding_lookup(embedding, [1, 2, 3, 4]).eval()\n'